In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

/home/chenming/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    # cache_dir="~/.cache/huggingface/hub"
)

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA GeForce RTX 2070. Max memory: 8.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import re
from pathlib import Path
from datasets import Dataset

train_data = []
training_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
你是一个生物信息学家，现在你要根据用户提供的物种，组织类型和差异基因来预测单细胞转录组数据的cluster最可能的细胞类型
如果用户输入的差异基因列表与marker 基因列表 {marker_genes} 相似，则返回细胞类型{celltype}及其OBO Ontology ID：{oboid}

### Input：
--------------------
物种： 人
组织类型：{tissue}
定义范围：{celltype_level}
差异基因列表：{marker_genes}
---------------------

### Response:
预测细胞类型：{celltype}
OBO Ontology ID: {oboid}
"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

for f in Path("./Azimuth_cellmarkers").iterdir():
    re_str = re.search(r'^(?P<tissue>\w+)_(?P<level>\w+).txt$',f.name)
    if re_str:
        tissue = re_str.group('tissue')
        level = re_str.group('level')
        print(f"{tissue}:{level}")
        with f.open("r") as indata:
            title = indata.readline()
            for line in indata:
                line_list = line.strip().split("\t")
                label,celltype,oboid,marker_genes = line_list
                celltype_level = "细胞大类" if level == "level1" else "细胞小类"
                cast_dict = {
                    "tissue":tissue,
                    "celltype_level":celltype_level,
                    "celltype":celltype,
                    "oboid":oboid,
                    "marker_genes":marker_genes,
                }

                # 格式化模板
                formatted_prompt = training_prompt.format(**cast_dict) + EOS_TOKEN 
                 # 将结果添加到训练数据列表
                train_data.append({"prompt": formatted_prompt})

# 创建 Dataset 对象
dataset = Dataset.from_dict({"text": [item["prompt"] for item in train_data]})

# 输出以确认数据格式
print(dataset)


pbmc:level1
pbmc:level2
Dataset({
    features: ['text'],
    num_rows: 37
})


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.

    args = TrainingArguments(
        per_device_train_batch_size = 1, # default 2
        gradient_accumulation_steps = 8, # default 4
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = True,
        bf16 = False,
        # fp16 = not torch.cuda.is_bf16_supported(),
        # bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2): 100%|██████████| 37/37 [00:00<00:00, 56.50 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [6]:
if torch.cuda.is_available():
    gpu_stats = torch.cuda.get_device_properties(0)
    allocated_gpu_memory = round(torch.cuda.memory_allocated(0) / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
    print(f"{allocated_gpu_memory} GB of memory currently in use.")
else:
    print("CUDA is not available on this system.")

GPU = NVIDIA GeForce RTX 2070. Max memory = 8.0 GB.
5.984 GB of memory currently in use.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 37 | Num Epochs = 15
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.377300
2,2.376500
3,2.350800
4,2.196200
5,2.013800
6,1.776900
7,1.481700
8,1.253300
9,0.965100
10,0.724900


/home/chenming/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /unsloth/meta-llama-3.1-8b-bnb-4bit/resolve/main/config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f7988d59890>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: 4759c692-dfa6-42f9-b79b-3dbad90994bc)') - silently ignoring the lookup for the file config.json in unsloth/meta-llama-3.1-8b-bnb-4bit.
  warnings.warn(
/home/chenming/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in unsloth/meta-llama-3.1-8b-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


In [8]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - allocated_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

439.658 seconds used for training.
7.33 minutes used for training.
Peak reserved memory = 6.586 GB.
Peak reserved memory for training = 0.602 GB.
Peak reserved memory % of max memory = 82.325 %.
Peak reserved memory for training % of max memory = 7.525 %.


In [15]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)

inputs = tokenizer([
"""
### Instruction:
你是一个生物信息学家，现在你要根据用户提供的物种，组织类型和差异基因来预测单细胞转录组数据的cluster最可能的细胞类型
如果用户输入的差异基因列表与marker 基因列表相似，则返回细胞类型及其OBO Ontology ID
最后还需要分别给出用户输入的差异基因列表中每个基因的功能

### Input:
--------------------
物种： 人
组织类型：PBMC
定义范围：细胞小类
差异基因列表：IL7R, TMSB10, CD4, ITGB1

---------------------
请给出预测的细胞类型

### Response:

"""], return_tensors = "pt").to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>
### Instruction:
你是一个生物信息学家，现在你要根据用户提供的物种，组织类型和差异基因来预测单细胞转录组数据的cluster最可能的细胞类型
如果用户输入的差异基因列表与marker 基因列表相似，则返回细胞类型及其OBO Ontology ID
最后还需要分别给出用户输入的差异基因列表中每个基因的功能

### Input:
--------------------
物种： 人
组织类型：PBMC
定义范围：细胞小类
差异基因列表：IL7R, TMSB10, CD4, ITGB1

---------------------
请给出预测的细胞类型

### Response:

预测细胞类型：CD4+ Central Memory T及其OBO Ontology ID：CD4-positive, alpha-beta memory T cell
定义范围:细胞小类
功能:CD4-positive, alpha-beta memory T cell
<|end_of_text|>


In [12]:
FastLanguageModel.for_inference(model)

inputs = tokenizer([
"""
### Instruction:
你是一个生物信息学家，现在你要根据用户提供的物种，组织类型和差异基因来预测单细胞转录组数据的cluster最可能的细胞类型
如果用户输入的差异基因列表与marker 基因列表相似，则返回细胞类型及其OBO Ontology ID

### Input:
--------------------
物种： 人
组织类型：PBMC
定义范围：细胞小类
差异基因列表：CTSS, FCN1, NEAT1, LYZ

---------------------
请给出预测的细胞类型

### Response:

"""], return_tensors = "pt").to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>
### Instruction:
你是一个生物信息学家，现在你要根据用户提供的物种，组织类型和差异基因来预测单细胞转录组数据的cluster最可能的细胞类型
如果用户输入的差异基因列表与marker 基因列表相似，则返回细胞类型及其OBO Ontology ID

### Input:
--------------------
物种： 人
组织类型：PBMC
定义范围：细胞小类
差异基因列表：CTSS, FCN1, NEAT1, LYZ

---------------------
请给出预测的细胞类型

### Response:

预测细胞类型：CD14+ Monocyte
OBO Ontology ID: CD14-positive, CD16-negative classical monocyte
<|end_of_text|>


In [16]:
model.save_pretrained("azimuth_lora_model") # Local saving
tokenizer.save_pretrained("azimuth_lora_model")

/home/chenming/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /unsloth/meta-llama-3.1-8b-bnb-4bit/resolve/main/config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f786b170c90>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: b9c5cd47-89cb-4490-bc3f-de6addca1b97)') - silently ignoring the lookup for the file config.json in unsloth/meta-llama-3.1-8b-bnb-4bit.
  warnings.warn(
/home/chenming/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in unsloth/meta-llama-3.1-8b-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


('azimuth_lora_model/tokenizer_config.json',
 'azimuth_lora_model/special_tokens_map.json',
 'azimuth_lora_model/tokenizer.json')